In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install xlsxwriter
import pandas as pd
import numpy as np

# Ielādē apvienoto CSV failu (visu sezonu xG dati)
df = pd.read_csv("/content/drive/MyDrive/bakalauraDarbs/Premier_League_All_Seasons_xG.csv")

# Nodrošina, ka datumi ir datetime formātā
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Atfiltrē tikai spēles līdz izvēlētam datumam
current_date = pd.Timestamp("2025-05-24")
df = df[df["Date"] < current_date]

# Svaru piešķiršana dažādiem periodiem
last_10_weight = 0.4       # Svars pēdējām 10 spēlēm
season_weight = 0.55       # Svars pārējai šīs sezonas daļai
historical_weight = 0.05   # Svars iepriekšējiem gadiem


# Aprēķina katras komandas svaroto xG (uzbrukums) un xGc (aizsardzība), izmantojot pēdējo 10 spēļu,
# pārējās sezonas un vēsturisko spēļu vidējos ar attiecīgiem svariem.
def calculate_weighted_xG(df):
    weighted_xG = {}
    # Izveido unikālu komandu kopu no mājinieku un viesu sarakstiem
    teams = set(df["Home"].unique()).union(set(df["Away"].unique()))

    for team in teams:
        # Atlasām spēles, kur komanda ir mājinieki vai viesi
        team_home_games = df[df["Home"] == team]
        team_away_games = df[df["Away"] == team]

        # Sakārtojam pēc datuma no svaigākās spēles uz vecāko
        team_home_games = team_home_games.sort_values("Date", ascending=False)
        team_away_games = team_away_games.sort_values("Date", ascending=False)

        # Atlasām pēdējās 10 mājas un viesu spēles
        last_10_home = team_home_games[:10]
        last_10_away = team_away_games[:10]

        # Atlasām pārējās (vecākās) šīs sezonas spēles nākamajiem svariem
        season_home = team_home_games[10:]
        season_away = team_away_games[10:]

        # Atlasām vēsturiskos (iepriekšējo sezonu) mačus, t.i., ne pašreizējā gada
        historical_home = df[(df["Home"] == team) & (~df["Date"].dt.year.isin([current_date.year]))]
        historical_away = df[(df["Away"] == team) & (~df["Date"].dt.year.isin([current_date.year]))]

        # Aprēķinām svaroto vidējo xG un xGconceded vērtības
        xG_Home = (
            last_10_home["xG_Home"].mean() * last_10_weight +
            season_home["xG_Home"].mean() * season_weight +
            historical_home["xG_Home"].mean() * historical_weight
        )
        xG_Away = (
            last_10_away["xG_Away"].mean() * last_10_weight +
            season_away["xG_Away"].mean() * season_weight +
            historical_away["xG_Away"].mean() * historical_weight
        )
        xG_Conceded_Home = (
            last_10_home["xG_Away"].mean() * last_10_weight +
            season_home["xG_Away"].mean() * season_weight +
            historical_home["xG_Away"].mean() * historical_weight
        )
        xG_Conceded_Away = (
            last_10_away["xG_Home"].mean() * last_10_weight +
            season_away["xG_Home"].mean() * season_weight +
            historical_away["xG_Home"].mean() * historical_weight
        )

        # Izveidojam vārdnīcu ar visām nozīmīgajām xG un xGc vērtībām konkrētai komandai
        weighted_xG[team] = {
            "xG": (xG_Home + xG_Away) / 2,            # Kopējais vidējais xG (uzbrukums)
            "xGAway": xG_Away,                        # Uzbrukuma xG kā viesiem
            "xGHome": xG_Home,                        # Uzbrukuma xG mājās
            "xGc": (xG_Conceded_Home + xG_Conceded_Away) / 2,   # xG ielaistie (aizsardzība)
            "xGcAway": xG_Conceded_Away,              # xGc kā viesiem
            "xGcHome": xG_Conceded_Home,              # xGc mājās
        }

    # Pārvēršam vārdnīcu DataFrame formātā
    return pd.DataFrame.from_dict(weighted_xG, orient="index")

# Aprēķinām xG un xGc reitingus visām komandām
ranking_df = calculate_weighted_xG(df)

# Aprēķinām visas līgas mediānu uzbrukumam un aizsardzībai
median_offense = ranking_df["xG"].median()
median_defense = ranking_df["xGc"].median()

# Komandas stipruma aprēķins:
# Kā komanda attiecas no līgas medianas uzbrukumā un aizsardzībā, lielāka vērtība = spēcīgāka komanda
ranking_df["TeamStrength"] = (ranking_df["xG"] - median_offense) - (ranking_df["xGc"] - median_defense)

# 2024/2025 sezonas komandu saraksts (pielāgo, ja vajag citu sezonu)
premier_league_teams_2024_2025 = [
    "Liverpool", "Manchester City", "Arsenal", "Chelsea", "Newcastle Utd",
    "Brighton", "Crystal Palace", "Bournemouth", "Brentford", "Fulham",
    "Aston Villa", "Manchester Utd", "Tottenham", "Everton", "Nott'ham Forest",
    "West Ham", "Wolves", "Leicester City", "Southampton", "Ipswich Town"
]

# Atlasām tikai šīs sezonas komandas rezultātu tabulā
ranking_df = ranking_df[ranking_df.index.isin(premier_league_teams_2024_2025)]

# Izvada tabulas: Vispirms aizsardzības, tad uzbrukuma, tad komandas stipruma reitings
print("Defensive Ranking:")  # Aizsardzības reitings pēc ielaistā xG
print(ranking_df[["xGc", "xGcAway", "xGcHome"]].sort_values(by="xGc"))

print("\nOffensive Ranking:")  # Uzbrukuma reitings pēc gūtā xG
print(ranking_df[["xG", "xGAway", "xGHome"]].sort_values(by="xG", ascending=False))

print("\nTeam Strength Table:")  # Kopējais komandas stiprums
print(ranking_df[["TeamStrength"]].sort_values(by="TeamStrength", ascending=False))

Defensive Ranking:
                      xGc   xGcAway   xGcHome
Manchester City  0.969496  1.020404  0.918588
Arsenal          0.999989  1.070566  0.929412
Liverpool        1.081025  1.224000  0.938050
Chelsea          1.126832  1.289900  0.963765
Crystal Palace   1.246711  1.378534  1.114889
Brighton         1.267552  1.372987  1.162118
Newcastle Utd    1.304510  1.389419  1.219602
Everton          1.328465  1.477401  1.179529
Brentford        1.352547  1.298911  1.406182
Aston Villa      1.368298  1.503279  1.233317
Fulham           1.379203  1.492364  1.266042
Manchester Utd   1.439626  1.540235  1.339017
Bournemouth      1.447522  1.573960  1.321083
Nott'ham Forest  1.471251  1.690740  1.251762
West Ham         1.496459  1.561571  1.431347
Wolves           1.502724  1.625493  1.379955
Tottenham        1.530445  1.491059  1.569832
Leicester City   1.643818  1.754117  1.533520
Southampton      1.863770  1.928935  1.798605
Ipswich Town     1.939146  2.138667  1.739625

Offensive Rank

In [6]:
save_path_excel = "/content/drive/My Drive/bakalauraDarbs/team_rankings_all.xlsx"

with pd.ExcelWriter(save_path_excel, engine="xlsxwriter") as writer:
    # Defensive Ranking lapa
    defensive = ranking_df[["xGc", "xGcAway", "xGcHome"]].sort_values(by="xGc")
    defensive.to_excel(writer, sheet_name="Defensive Ranking")
    # Offensive Ranking lapa
    offensive = ranking_df[["xG", "xGAway", "xGHome"]].sort_values(by="xG", ascending=False)
    offensive.to_excel(writer, sheet_name="Offensive Ranking")
    # Team Strength Table lapa
    team_strength = ranking_df[["TeamStrength"]].sort_values(by="TeamStrength", ascending=False)
    team_strength.to_excel(writer, sheet_name="Team Strength Table")

print(f"Excel fails saglabāts šeit: {save_path_excel}")

Excel fails saglabāts šeit: /content/drive/My Drive/bakalauraDarbs/team_rankings_all.xlsx
